## Tolman catalyst structure generation

The following notebook automatically generates the nickel phosphine ligated tricarbonyl catalysts used to calculate the Tolman electronic parameter. To ease the generation of the structures, the notebook uses the ligand structures already generated when making the palladium complexes.

In [1]:
import molli

molli.visual.configure(bgcolor="1F1F1F")

catalyst = molli.load("data/dft/qsar/benzylamine/submission/Ni(CO)3.mol2")

In [ ]:
from pathlib import Path

STRUCGEN_ROOT = Path("data/dft/qsar/benzylamine/struc_gen")

lowest_energy_structures = sorted(list(STRUCGEN_ROOT.glob("*/featurize_ensemble-long/STRUC1/opt.xtbopt.xyz")))

ligands_mlib = molli.MoleculeLibrary(
    "data/dft/qsar/benzylamine/submission/ligands-tolman.mlib", readonly=False, overwrite=True
)

with ligands_mlib.writing():
    for struc in lowest_energy_structures:
        mol = molli.load(struc, parser="openbabel")
        mol.name = struc.parents[2].name

        ligand = mol.substructure([0] + list(range(76, len(mol.atoms))))
        ligand.name = mol.name

        ligand.get_atom(molli.Element.Pd).label = "AP1"

        ligand = molli.Molecule(ligand)

        tolman_mol = molli.Molecule.join(
            catalyst,
            ligand,
            "AP1",
            "AP1",
            optimize_rotation=True,
            name=mol.name,
        )

        workdir = STRUCGEN_ROOT / tolman_mol.name / "tolman"
        workdir.mkdir(parents=True, exist_ok=True)

        with open(workdir / "geom.xyz", "w") as f:
            tolman_mol.dump_xyz(f)